<H1>Import Libraries</H1>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
# from sklearn.metrics import confusion_matrix
from torchvision.datasets import CIFAR10
from collections import Counter
import random
import copy
import torch.nn.functional as F
import os
import sys
import time
import pickle
import pandas as pd
from torch import Tensor
from typing import Type
import torchvision.models as models
from torchvision import transforms

# Set font family for plots
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


<H1>Load Dataset</H1>

In [2]:
# transform = transforms.Compose([
#     transforms.ToTensor(), #This transformation converts a PIL (Python Imaging Library) Image or numpy.ndarray (with shape (H x W x C) in the range [0, 255]) into a PyTorch tensor of shape (C x H x W) in the range [0.0, 1.0]. It essentially rearranges the dimensions of the image data and scales it to a float value between 0 and 1.
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
# ])


# train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
# # train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

# test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

subset_dataset = torch.load('val_dataset.pth',weights_only=False)
remaining_dataset = torch.load('test_dataset.pth',weights_only=False)

# Create DataLoaders
val_loader = DataLoader(subset_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(remaining_dataset, batch_size=32, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


<H1>IID</H1>

In [3]:
def distribute_dataset_equally(dataset, num_clients):
    # Group data by class
    class_data = {}
    for data, label in dataset:
        if label not in class_data:
            class_data[label] = []
        class_data[label].append((data, label))

    # Distribute data
    client_data = [[] for _ in range(num_clients)]
    for label, data in class_data.items():
        data_len = len(data)
        base_size = data_len // num_clients
        remain = data_len - base_size * num_clients

        current_idx = 0
        for i in range(num_clients):
            end_idx = current_idx + base_size + (1 if i < remain else 0)
            client_data[i].extend(data[current_idx:end_idx])
            current_idx = end_idx


    print_iid_distribution(client_data)
    plot_iid_dataset(client_data,num_clients)

    return client_data

In [4]:
#print client iid_data distribution
def print_iid_distribution(iid_datasets):
    # Check if the distribution is correct
    for i, client_data in enumerate(iid_datasets):
        print(f"Client {i + 1} data size: {len(client_data)}")
        class_counts = {j: 0 for j in range(10)}
        for _, label in client_data:
            class_counts[label] += 1
        print(f"Class distribution: {class_counts}")

In [5]:
# Plot client iid_data distribution
def plot_iid_dataset(iid_datasets, num_clients):
    # CIFAR-10 class names
    class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
    # Calculate fraction of distribution for each class across clients
    fractions = []
    for client_data in iid_datasets:
        class_counts = {j: 0 for j in range(10)}
        for _, label in client_data:
            class_counts[label] += 1
        total_data = len(client_data)
        fractions.append([class_counts[i] / total_data for i in range(10)])
    fractions = np.array(fractions)
    # Define colors for each class
    colors = ["r", "g", "b", "c", "m", "y", "#FFA500", "#FF00FF", "#808080", "#00FF00"]
    # Generate the 3D bar chart
    fig = plt.figure(figsize=(10, 7))  # Adjust the size here
    ax = fig.add_subplot(111, projection='3d')

    xpos, ypos = np.meshgrid(np.arange(num_clients), np.arange(10), indexing="ij")
    for i in range(10):
        ax.bar3d(xpos[:, i], ypos[:, i], np.zeros_like(xpos[:, i]),0.75, 0.75, fractions[:, i],shade=True, color=colors[i])

    ax.set_xlabel('Client Number',labelpad=5)
    ax.set_ylabel('CIFAR-10 Classes',labelpad=15)
    ax.set_zlabel('Fraction of Distribution',labelpad=5)
    ax.set_xticks(np.arange(0.5, num_clients))
    ax.set_xticklabels([str(i+1) for i in range(num_clients)], rotation=45)
    ax.set_yticks(np.arange(0.5, 10))
    ax.set_yticklabels(class_names, rotation=-60)

    #ax.view_init(elev=40, azim=60)
    ax.view_init(elev=40, azim=10)

    plt.subplots_adjust(left=0.01, right=0.99, bottom=0.01, top=0.99)

    plt.show()

<H1>non-IID</H1>

In [6]:
#Print the distribution non IID
def print_distribution(client_data):
    # Check distribution
    for l, client_data_value in enumerate(client_data):
        print(f"Client {l + 1} data size: {len(client_data_value)}")
        class_counts = {j: 0 for j in range(10)}
        for _, label in client_data_value:
            class_counts[label] += 1
        print(f"Class distribution: {class_counts}")

In [7]:
# #Non IID code
def distribute_dataset_dirichlet(dataset, num_clients, alpha):
    # Group data by class
    class_data = {}
    for data, label in dataset:
        if label not in class_data:
            class_data[label] = []
        class_data[label].append((data, label))

    client_data = [[] for _ in range(num_clients)]
    for _, data_list in class_data.items():
        # Shuffle data for randomness
        np.random.shuffle(data_list)

        # Get proportions for data split based on Dirichlet distribution
        proportions = np.random.dirichlet([alpha]*num_clients)
        # print("Proportions: ", proportions)
        total_data = len(data_list)
        # print("total_data: ", total_data)
        data_splits = [int(proportions[i]*total_data) for i in range(num_clients)]
        # print("Data_Split: ", data_splits)

        # Adjust the splits to account for rounding errors
        # print("Before data_split:",data_splits)
        data_splits[-1] += total_data - sum(data_splits)
        print("After data_split:",data_splits)

        start_idx = 0
        for i, split in enumerate(data_splits):
            end_idx = start_idx + split
            client_data[i].extend(data_list[start_idx:end_idx])
            start_idx = end_idx
    print("Client Number: ",num_clients, "Alpha: ", alpha)
    print_distribution(client_data)
    plot_distribution(client_data, num_clients )
    return client_data

In [8]:
#show the graphs for nonIID
def plot_distribution(client_datasets, num_clients):
    # CIFAR-10 class names
    class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
    # Calculate fraction of distribution for each class across clients
    fractions = []
    for client_data in client_datasets:
        class_counts = {j: 0 for j in range(10)}
        for _, label in client_data:
            class_counts[label] += 1
        total_data = len(client_data)
        # print(total_data)
        fractions.append([class_counts[i] / total_data for i in range(10)])
    fractions = np.array(fractions)
    # Define distinct colors for classes using a colormap
    colors = ["r", "g", "b", "c", "m", "y", "#FFA500", "#FF00FF", "#808080", "#00FF00"]
    # # Generate the 3D bar chart
    # fig = plt.figure(figsize=(18, 12))
    # ax = fig.add_subplot(111, projection='3d')

    # Generate the 3D bar chart
    fig = plt.figure(figsize=(10, 7))  # Adjust the size here
    ax = fig.add_subplot(111, projection='3d')

    xpos, ypos = np.meshgrid(np.arange(num_clients), np.arange(10), indexing="ij")

    for i in range(10):
        ax.bar3d(xpos[:, i], ypos[:, i], np.zeros_like(xpos[:, i]),0.75, 0.75, fractions[:, i],shade=True, color=colors[i])

    ax.set_xlabel('Client Number')
    ax.set_ylabel('CIFAR-10 Classes')
    ax.set_zlabel('Fraction of Distribution')
    ax.set_title('Distribution of CIFAR-10 Classes across Clients based on Dirichlet Distribution')
    ax.set_xticks(np.arange(0.5, num_clients))
    ax.set_xticklabels([str(i+1) for i in range(num_clients)], fontsize=12)
    ax.set_yticks(np.arange(0.5, 10))
    ax.set_yticklabels(class_names)
    plt.tight_layout()
    plt.show()

# Measures

In [9]:
def accuracy(outp, target):
    """Computes accuracy"""
    with torch.no_grad():
        pred = torch.argmax(outp, dim=1)
        correct = pred.eq(target).float().sum().item()
        return 100.0 * correct / target.size(0)

In [10]:
def Print(string, dictionary):
    first_key = next(iter(dictionary))
    first_value = dictionary[first_key]
    print(f"{string}:{first_key}: {first_value[0][0]}\n")

In [11]:
def forbinus_norm_function(w_i):
    value = 0
    for k in w_i.keys():
        value += torch.linalg.norm(w_i[k])
    return value.item()

In [12]:
def model_deviation_function(w_i, w_f):
    model_deviation = 0
    for k in w_i.keys():
        model_deviation += torch.linalg.norm(w_f[k].to(torch.float) - w_i[k].to(torch.float)) / torch.linalg.norm(w_i[k].to(torch.float))
    #print(model_deviation.item())
    return model_deviation.item()

# Model

In [13]:
class model(nn.Module):
    def __init__(self):
        super(model, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(0.5)
        
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.relu4 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout2 = nn.Dropout(0.5)

        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.relu5 = nn.ReLU()
        self.dropout3 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 10)  # Change output to 10 for CIFAR-10 classes

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.pool1(x)
        x = self.dropout1(x)
        x = self.relu3(self.conv3(x))
        x = self.relu4(self.conv4(x))
        x = self.pool2(x)
        x = self.dropout2(x)
        x = x.view(x.size(0), -1)
        x = self.relu5(self.fc1(x))
        x = self.dropout3(x)
        x = self.fc2(x)
        return x

# Train

In [14]:
def average_weights_function(dicts):
    # Initialize an empty dictionary to store the summed weights
    summed_weights = {}
    
    # Initialize an empty dictionary to keep track of how many times each key is seen
    key_occurrences = {}
    
    # Iterate through all the dictionaries
    for d in dicts:
        for key, value in d.items():
            if key in summed_weights:
                summed_weights[key] += value
                key_occurrences[key] += 1
            else:
                summed_weights[key] = value
                key_occurrences[key] = 1
    
    # Create a dictionary to store the final averaged weights
    averaged_weights = {}
    
    # Iterate through the summed weights and divide by the number of occurrences for each key
    for key, value in summed_weights.items():
        if key_occurrences[key] > 1:
            averaged_weights[key] = value / key_occurrences[key]
        else:
            averaged_weights[key] = value  # If only present in one dict, keep as is
    
    return averaged_weights

In [15]:
def train(i_weights, epochs, train_loader, le_rate, cli,roun, epoch_flag, ci,c):
    global opti
    
    local_model = model().to(device)
    criterion = nn.CrossEntropyLoss()
    if opti=="adam":
        optimizer = torch.optim.Adam(local_model.parameters(), lr=le_rate)
    elif opti=="sgd":
        optimizer = torch.optim.SGD(local_model.parameters(), lr=le_rate)
    
    epoch_train_accuracy=0 
    epoch_train_loss=0
    epoch_test_accuracy=0
    epoch_test_loss=0
    epoch_rmd=0

    local_model.load_state_dict(i_weights)

    # Initialize local model with the global model weights
    yi = {k: v.clone().float().to(device) for k, v in i_weights.items()}
    # Ensure control variates are float
    ci = {k: v.float().to(device) for k, v in ci.items()}
    c = {k: v.float().to(device) for k, v in c.items()}
    
    # Adjust gradients with control variate (c - ci)
    with torch.no_grad():  # Ensure no gradient tracking for this operation
        for k in local_model.state_dict().keys():
            param = local_model.state_dict()[k].float()
            local_model.state_dict()[k] = param - (ci[k] - c[k])

    local_model.train()  # Set the model to training mode

    # initial weights cathing and printing
    initial_weights = {k: v.clone() for k, v in local_model.state_dict().items()}
    #Print("Model's inside the function Initial weights for client",initial_weights)

    # Training loop
    for epoch in range(epochs):
        epoch_flag=epoch_flag+1
        # gradients_this_epoch = {}
        total_samples = 0
        total_loss=0
        correct_samples = 0
        for i, (inputs, labels) in enumerate(train_loader, 0):
            inputs, labels = inputs.to(device), labels.to(device)
            # Zero the parameter gradients
            optimizer.zero_grad()
            # Forward + backward + optimize
            outputs = local_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            total_loss += loss.item()
            optimizer.step()

            _, predicted = outputs.max(1)  # Get the index of the maximum value in outputs (predicted class)
            total_samples += labels.size(0)
            correct_samples += predicted.eq(labels).sum().item()
        
        if(total_samples!=0 and len(train_loader)!=0):
            epoch_accuracy = 100 * correct_samples / total_samples
            epoch_loss = total_loss / len(train_loader)
        else:
            epoch_accuracy = 100 * correct_samples / (total_samples+1)
            epoch_loss = total_loss / (len(train_loader)+1)
        print(f"Round {roun}, cleint {cli+1}, epoch {epoch+1}: epoch_accuracy {epoch_accuracy}, epoch_loss {epoch_loss} ")
    
    f_weights = {k: v.clone() for k, v in local_model.state_dict().items()}

    #print(f"\n Round {roun}, cleint {cli}: epoch_accuracy {epoch_accuracy}, epoch_loss {epoch_loss} \n")
    epoch_train_accuracy=epoch_accuracy
    epoch_train_loss=epoch_loss
    epoch_test_accuracy, epoch_test_loss= test(f_weights, test_loader)
    
    
    epoch_rmd=model_deviation_function(initial_weights,f_weights)
    
    #saving data into dataframe
    epoch_data = [epoch_train_accuracy, epoch_train_loss, epoch_test_accuracy, epoch_test_loss, epoch_rmd]
    epoch_results.loc[len(epoch_results)] = epoch_data

    # Update client control variate ci using Option II
    new_ci = {
        k: ci[k] - c[k] + (1 / (epochs * le_rate)) * (i_weights[k].float() - f_weights[k])
        for k in ci.keys()
    }
    return epoch_accuracy,epoch_loss, f_weights, epoch_flag, new_ci

# Test

In [16]:
def test(w,data):
    lmodel = model().to(device)
    criterion = nn.CrossEntropyLoss()  # Assuming a classification task
    #optimizer = torch.optim.SGD(lmodel.parameters(), lr=learning_rate)
    lmodel.load_state_dict(w)
    lmodel.eval()

    #checking the weights
    tw = lmodel.state_dict()
    #Print("Model's before testing the weights in global model",tw)

    # Evaluation phase for test set
    acc_list = []
    loss_list = []

    with torch.no_grad():
        for j, data in enumerate(data, 0):
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            out = lmodel(images)
            # Calculate loss
            loss = criterion(out, labels)
            loss_list.append(loss.item())
            #calculate accuracy
            acc = accuracy(out, labels)
            acc_list.append(acc)
    test_loss = np.mean(loss_list)
    test_accuracy = np.mean(acc_list)
    #print("Model's Test accuracy : {:.2f}%".format(test_accuracy))
    return test_accuracy, test_loss

# FL Structure

In [17]:
def federated_learning(i_w, data_client, C, P, R, E, learning_rate, b_size):
    
    global total_clients_list, participating_client_list
    
    global_model.load_state_dict(i_w)
    #Print("Model's initial weights", i_w)

    c_list = {k: torch.zeros_like(v, dtype=torch.float).to(device) for k, v in i_w.items()}  # Initialize global control variate to zero
    ci_list = [{k: torch.zeros_like(v, dtype=torch.float).to(device) for k, v in i_w.items()} for _ in range(C)]  # Initialize client control variates to zero

    #loop for round
    for r in range(1,R+1):
        round_train_accuracy=0
        round_train_loss=0
        round_test_accuracy=0
        round_test_loss=0
        epoch_flag=0

        models = []

        #saving initial weights for spiking model
        i_w = {k: v.clone() for k, v in global_model.state_dict().items()}
        #Print("Model's initial weights", i_w)
        
        #colleting weights and results
        all_final_weights={}
        train_accuracy_list=[]
        train_loss_list=[]
        
        # Randomly select clients
        selected_clients = random.sample(total_clients_list, P)
        participating_client_list.append(selected_clients)

        #loop for client
        for c, data in enumerate(data_client):
            
            if(c in selected_clients):
                
                c_i = ci_list[c] 
                
                train_loader = torch.utils.data.DataLoader(data, batch_size=b_size, shuffle=True, drop_last=True)
                
                #train model
                train_accuracy, train_loss, c_f_weights, epoch_flag, new_ci = train(i_w, E, train_loader, learning_rate, c, r,epoch_flag, c_i, c_list)

                models.append( c_f_weights)

                ci_list[c] = new_ci

                train_accuracy_list.append(train_accuracy)
                train_loss_list.append(train_loss)

                # Accumulate weights for the selected client
                #for param_name, param_grad in c_f_weights.items():
                    #if param_name in all_final_weights:
                        #all_final_weights[param_name] += param_grad
                    #else:
                        #all_final_weights[param_name] = param_grad

            else:
                print(f"client {c+1} is not selectecd")
                
        all_final_weights = average_weights_function(models)
        
        round_epoch=(epoch_flag)

        for k in c_list.keys():
            # Convert to float before summing and assigning
            ci_sum = sum([ci_list[n_no][k].float() for n_no in range(C)])
            c_list[k] += (1 / C) * ci_sum
        
        #print("Total number of selected clients is", client_counter)
        round_train_loss=sum(train_loss_list)/len(train_loss_list)
        round_train_accuracy=sum(train_accuracy_list)/len(train_accuracy_list)

        print(f"Model's Round: {r}, train accuracy of model: {round_train_accuracy}, train loss of model: {round_train_loss} \n\n")

        #for param_name in all_final_weights:
            #all_final_weights[param_name] = all_final_weights[param_name].float() / len(selected_clients)

        round_test_accuracy, round_test_loss=test(all_final_weights, test_loader)
        print(f"Model's Round: {r}, test accuracy of model: {round_test_accuracy}, test loss of model: {round_test_loss} \n\n")

        #model deviation code
        round_rmd=model_deviation_function(i_w, all_final_weights)
        #print("Model deviation values: ", model_deviation)

        #saving data into dataframe
        round_data = [round_train_accuracy, round_train_loss, round_test_accuracy, round_test_loss, round_rmd, round_epoch]
        round_results.loc[len(round_results)] = round_data
            
        global_model.load_state_dict(all_final_weights)
        print("round", r, "completed")

# Main Function

# Define parameters

In [18]:
#===========================Parameters==============================================================
client_no=20
participating_client=20
epochs=5
learning_rate=0.0001
round_no=30
batch_size=128
distributions = "non_iid" # 'non_iid'
data_class=10
alpha=0.5
opti="sgd" # or SGD

method="fed_scaffold"

# List of clients
total_clients_list = list(range(0, client_no))
# print(total_cleints_list)
participating_client_list=[]

# Define dataframe for round results
round_columns = ['train_accuracy', 'train_loss', 'test_accuracy', 'test_loss', 'rmd', 'epoch']
round_results = pd.DataFrame(columns=round_columns)

# Define dataframe for epoch results
epoch_columns = ['train_accuracy', 'train_loss', 'test_accuracy', 'test_loss', 'rmd']
epoch_results = pd.DataFrame(columns=epoch_columns)

#===================================loading the saved weight list====================================================
global_model = model().to(device)
# initial_weights={k: v.clone() for k, v in global_model.state_dict().items()}
# Save the initial weights
file_path = "s_cnn.pth"
# torch.save(initial_weights, file_path)
initial_weights=torch.load(file_path,weights_only=True)
Print("Model's initial weights", initial_weights)


Model's initial weights:conv1.weight: tensor([[-0.0619,  0.0628, -0.1613],
        [-0.1879,  0.0229,  0.1701],
        [-0.1736, -0.0053,  0.0203]], device='cuda:0')



<H1>Divide data among cleints</H1>

In [19]:
# #=================================loading IID data===========================
# if distributions == 'iid':
#     client_datasets = distribute_dataset_equally(train_dataset,client_no)
    
# elif distributions == 'non_iid':
#     client_datasets = distribute_dataset_dirichlet(train_dataset, client_no, alpha)
# else:
#     print("provide a valid distribution Please")

In [20]:
# # Save client_datasets to a file
# if distributions == 'iid':
#     with open('client_datasets_IID.pkl', 'wb') as f:
#         pickle.dump(client_datasets, f)
# elif distributions == 'non_iid' and alpha==0.5:
#     with open('client_datasets_non_IID_0_5.pkl', 'wb') as f:
#         pickle.dump(client_datasets, f)
# elif distributions == 'non_iid' and alpha==0.125:
#     with open('client_datasets_non_IID_0_125.pkl', 'wb') as f:
#         pickle.dump(client_datasets, f)

# print("client_datasets saved successfully.")

In [21]:
# Load client_datasets from a file
if distributions == 'iid':
    with open('20_client_datasets_IID.pkl', 'rb') as f:
        client_datasets = pickle.load(f)

elif distributions == 'non_iid' and alpha==0.5:
    with open('20_client_datasets_non_IID_0_5.pkl', 'rb') as f:
        client_datasets = pickle.load(f)
    
elif distributions == 'non_iid' and alpha==0.125:
    with open('20_client_datasets_non_IID_0_125.pkl', 'rb') as f:
        client_datasets = pickle.load(f)
        
print("client_datasets loaded successfully.")
print_distribution(client_datasets)

client_datasets loaded successfully.
Client 1 data size: 1928
Class distribution: {0: 93, 1: 405, 2: 144, 3: 3, 4: 342, 5: 365, 6: 25, 7: 64, 8: 248, 9: 239}
Client 2 data size: 1997
Class distribution: {0: 38, 1: 0, 2: 921, 3: 588, 4: 14, 5: 9, 6: 58, 7: 51, 8: 11, 9: 307}
Client 3 data size: 3557
Class distribution: {0: 849, 1: 41, 2: 2, 3: 208, 4: 645, 5: 640, 6: 853, 7: 151, 8: 20, 9: 148}
Client 4 data size: 1954
Class distribution: {0: 369, 1: 91, 2: 9, 3: 135, 4: 108, 5: 65, 6: 267, 7: 30, 8: 673, 9: 207}
Client 5 data size: 3332
Class distribution: {0: 25, 1: 0, 2: 866, 3: 1163, 4: 6, 5: 194, 6: 13, 7: 375, 8: 61, 9: 629}
Client 6 data size: 2673
Class distribution: {0: 241, 1: 643, 2: 161, 3: 26, 4: 2, 5: 861, 6: 65, 7: 182, 8: 462, 9: 30}
Client 7 data size: 2750
Class distribution: {0: 161, 1: 0, 2: 503, 3: 26, 4: 184, 5: 443, 6: 6, 7: 6, 8: 1412, 9: 9}
Client 8 data size: 2071
Class distribution: {0: 217, 1: 128, 2: 136, 3: 37, 4: 20, 5: 19, 6: 42, 7: 371, 8: 37, 9: 1064}
C

<H1>Round zero</H1>

In [22]:
#train accuracy for cleints
round_train_accuracy=0
round_train_loss=0

train_accuracy_list=[]
train_loss_list=[]
for c, data in enumerate(client_datasets):
    train_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, drop_last=True)
    train_accuracy, train_loss=test(initial_weights, train_loader)
    train_accuracy_list.append(train_accuracy)
    train_loss_list.append(train_loss)
round_train_accuracy=(sum(train_accuracy_list)/len(train_accuracy_list))
round_train_loss=(sum(train_loss_list)/len(train_loss_list))


#test accuracy for server
round_test_accuracy=0
round_test_loss=0
test_accuracy,test_loss=test(initial_weights,test_loader)
round_test_accuracy=(test_accuracy)
round_test_loss=(test_loss)

round_rmd=0
round_epoch=0

round_data = [round_train_accuracy, round_train_loss, round_test_accuracy, round_test_loss, round_rmd, round_epoch]
round_results.loc[len(round_results)] = round_data

Print("initial_weights", initial_weights)
print(f' train accuracy: {round_train_accuracy}\n train_loss: {round_train_loss}\n test_accuracy: {round_test_accuracy}\n test_loss: {round_test_loss}')

initial_weights:conv1.weight: tensor([[-0.0619,  0.0628, -0.1613],
        [-0.1879,  0.0229,  0.1701],
        [-0.1736, -0.0053,  0.0203]], device='cuda:0')

 train accuracy: 11.202316706095846
 train_loss: 2.300268226845141
 test_accuracy: 10.013977635782748
 test_loss: 2.3027472800720994


<H1>Run FL</H1>

In [23]:
federated_learning(initial_weights, client_datasets, client_no, participating_client, round_no, epochs, learning_rate, batch_size)

Round 1, cleint 1, epoch 1: epoch_accuracy 1.9270833333333333, epoch_loss 2.3075278759002686 
Round 1, cleint 1, epoch 2: epoch_accuracy 2.1354166666666665, epoch_loss 2.3073696454366046 
Round 1, cleint 1, epoch 3: epoch_accuracy 1.8229166666666667, epoch_loss 2.3071299393971763 
Round 1, cleint 1, epoch 4: epoch_accuracy 1.6145833333333333, epoch_loss 2.306867520014445 
Round 1, cleint 1, epoch 5: epoch_accuracy 1.6666666666666667, epoch_loss 2.3066601276397707 
Round 1, cleint 2, epoch 1: epoch_accuracy 28.958333333333332, epoch_loss 2.2672160466512046 
Round 1, cleint 2, epoch 2: epoch_accuracy 29.739583333333332, epoch_loss 2.265491278966268 
Round 1, cleint 2, epoch 3: epoch_accuracy 30.104166666666668, epoch_loss 2.2653316974639894 
Round 1, cleint 2, epoch 4: epoch_accuracy 30.416666666666668, epoch_loss 2.264072545369466 
Round 1, cleint 2, epoch 5: epoch_accuracy 30.15625, epoch_loss 2.2639458815256757 
Round 1, cleint 3, epoch 1: epoch_accuracy 5.3530092592592595, epoch_loss

# Reuslts

In [24]:
# Define the folder and file name
folder_name = f"{method}_{opti}_{learning_rate}_{participating_client}_{client_no}_{distributions}_{alpha}"  # Folder where the Excel file will be saved
file_name = "round_results.xlsx"


# Check if the folder exists, if not, create it
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Full path where the Excel file will be saved
file_path = os.path.join(folder_name, file_name)

round_results.to_excel(file_path, index=False)

print("DataFrame successfully written for round results.")

DataFrame successfully written for round results.


In [25]:
# Define the folder and file name
folder_name =  f"{method}_{opti}_{learning_rate}_{participating_client}_{client_no}_{distributions}_{alpha}"   # Folder where the Excel file will be saved
file_name = "epoch_results.xlsx"

# Check if the folder exists, if not, create it
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Full path where the Excel file will be saved
file_path = os.path.join(folder_name, file_name)

epoch_results.to_excel(file_path, index=False)

print("DataFrame successfully written for epoch results.")

DataFrame successfully written for epoch results.
